<a href="https://colab.research.google.com/github/26tiffanyz/Coral-Bleaching-Research/blob/main/Coral_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Reference https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
# Author: Tiffany Zhu
# Machine learning model that classifies coral images as either 'living' or 'dead' using transfer learning with pre-trained ResNet-50 model.

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

cudnn.benchmark = True
plt.ion() # interactive mode for matplotlib

In [1]:
# Set data directory path and mount Google Drive
data_dir = '/content/drive/My Drive/coral-data'
if 'drive' in data_dir:
  from google.colab import drive
  drive.flush_and_unmount()
  drive.mount('/content/drive')
  print('Mounted Google Drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive
Mounted Google Drive


In [ ]:
# Data augmentation and normalization for training
# Standardize the image size, brightness for training and validation sets
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets and create loaders
def load_datasets(datadir):
    return {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
            for x in ['train', 'val']}

image_datasets = load_datasets(data_dir)
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                             shuffle=True, num_workers=4)
              for x in ['train', 'val']}

# Get dataset sizes and class names
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
print("total images for training and validation", dataset_sizes)

# Set device, GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Show example of some training images, display images with normalization
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated



# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']), [None, None])
di = li = 0
while di < 10 and li < 10:
  for x in classes:
    if class_names[x] == 'living':
      li += 1
    else:
      di += 1

  # Make a grid from the batch
  out = torchvision.utils.make_grid(inputs)

  imshow(out, title=[class_names[x] for x in classes])
  inputs, classes = next(iter(dataloaders['train']), [None, None])
  print("Total training images of living and dead", li, di)


In [5]:
# Train the model and return best model based on validation accuracy
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over the data
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward pass
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward pass + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Track the statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            # Update the learning rate
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # Copy the best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    # Reporting the time it took to train
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [6]:
# Visualize the model predictions on validation set and show mislassifications
def visualize_model(model, num_images_to_show=20):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    # Counters for classification results
    num_living_correct = num_dead_correct = num_living_wrong = num_dead_wrong = 0

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            print("Total images: ", inputs.size()[0])

            # Looping through each image in the batch to count
            for j in range(inputs.size()[0]):
                images_so_far += 1
                pred = class_names[preds[j]]
                actual = class_names[labels[j]]
                if actual == 'dead':
                  if pred == 'dead':
                    num_dead_correct += 1
                  else:
                    num_dead_wrong += 1
                if actual == 'living':
                  if pred == 'living':
                    num_living_correct += 1
                  else:
                    num_living_wrong += 1

                # Display the wrongly classified images
                num_diffs = num_living_wrong + num_dead_wrong
                if (num_diffs < num_images_to_show and actual != pred):
                  plt.figure(figsize=(40, 40))
                  ax = plt.subplot(num_images_to_show//2, 2, num_diffs)
                  ax.axis('off')
                  ax.set_title(f'pred: {pred}, actual: {actual}')
                  imshow(inputs.cpu().data[j])

        print(f"num_living_correct: {num_living_correct}, num_living_wrong: {num_living_wrong}")
        print(f"num_dead_correct: {num_dead_correct}, num_dead_wrong: {num_dead_wrong}")

        # Bring back to original training state
        model.train(mode=was_training)


In [ ]:
# Fine-tuning a pre-trained ResNet-50 model
model_ft = models.resnet50(pretrained=True) # Load model
num_ftrs = model_ft.fc.in_features # Replace fully connected layer to match binary classification
model_ft.fc = nn.Linear(num_ftrs, 2) # 2 is num of output types

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss() # My loss function

# Optimizing all parameters for fine tuning
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)


In [ ]:
# Train model
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

In [ ]:
# Visualize results
visualize_model(model_ft)